# 📘 Step 2: 임베딩 및 Vector DB 저장 시스템

페이지별 텍스트를 문단 단위로 쪼개어 ChromaDB에 저장

## 사용 방법
1. 필요한 패키지 설치 (아래 셀 실행)
2. OpenAI API 키 설정 (환경변수 또는 직접 입력)
3. `pdf_preprocessor.ipynb`를 먼저 실행하여 데이터를 준비
4. 모든 셀을 순서대로 실행


In [ ]:
# 필요한 패키지 설치 (Google Colab용)
!pip install chromadb langchain langchain-openai tiktoken python-dotenv -q

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import json
import os
from typing import List, Dict, Any
import chromadb
from chromadb.config import Settings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import tiktoken
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()


In [ ]:
# OpenAI API 키 설정 (Colab에서 사용)
# 방법 1: 환경변수에서 로드 (권장)
import os
api_key = os.getenv('OPENAI_API_KEY')

# 방법 2: 직접 입력 (보안 주의!)
# api_key = "your-api-key-here"

if not api_key:
    print("⚠️ OPENAI_API_KEY가 설정되지 않았습니다.")
    print("💡 아래 셀에서 API 키를 설정하거나, 환경변수를 설정하세요.")
else:
    print("✅ OpenAI API 키가 설정되었습니다.")


In [ ]:
class VectorStore:
    def __init__(self, collection_name: str = "insurance_terms", api_key: str = None, chroma_db_path: str = "./chroma_db"):
        self.collection_name = collection_name
        
        # API 키 설정
        if api_key is None:
            api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
        
        self.embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)
        
        # ChromaDB 클라이언트 초기화 (경로 지정 가능)
        os.makedirs(chroma_db_path, exist_ok=True)
        self.client = chromadb.PersistentClient(
            path=chroma_db_path,
            settings=Settings(anonymized_telemetry=False)
        )
        
        # 컬렉션 생성 또는 가져오기
        try:
            self.collection = self.client.get_collection(name=collection_name)
            print(f"📚 기존 컬렉션 로드: {collection_name}")
        except:
            self.collection = self.client.create_collection(
                name=collection_name,
                metadata={"description": "보험 약관 문서 벡터 저장소"}
            )
            print(f"📚 새 컬렉션 생성: {collection_name}")
    
    def load_processed_data(self, json_file: str) -> List[Dict]:
        """처리된 JSON 데이터 로드"""
        print(f"📖 데이터 로드 중: {json_file}")
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"✅ {len(data)}페이지 데이터 로드 완료")
            return data
            
        except Exception as e:
            print(f"❌ 데이터 로드 오류: {str(e)}")
            return []
    
    def chunk_text(self, text: str, page: int, source: str) -> List[Dict]:
        """텍스트를 청크로 분할"""
        # 텍스트 분할기 설정
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,  # 300 토큰으로 줄임
            chunk_overlap=100,  # 100 토큰 오버랩으로 늘림
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )
        
        # 텍스트 분할
        chunks = text_splitter.split_text(text)
        
        # 청크 데이터 생성
        chunk_data = []
        for i, chunk in enumerate(chunks):
            if chunk.strip():  # 빈 청크 제외
                chunk_data.append({
                    "content": chunk.strip(),
                    "metadata": {
                        "page": page,
                        "source": source,
                        "chunk_id": i,
                        "total_chunks": len(chunks)
                    }
                })
        
        return chunk_data
    
    def process_all_pages(self, pages_data: List[Dict]) -> List[Dict]:
        """모든 페이지를 청크로 분할"""
        print("✂️ 텍스트 청킹 중...")
        
        all_chunks = []
        for page_data in pages_data:
            page_chunks = self.chunk_text(
                page_data["text"],
                page_data["page"],
                page_data["source"]
            )
            all_chunks.extend(page_chunks)
            
            print(f"   ✅ 페이지 {page_data['page']}: {len(page_chunks)}개 청크")
        
        print(f"✂️ 총 {len(all_chunks)}개 청크 생성 완료")
        return all_chunks
    
    def store_in_vector_db(self, chunks: List[Dict]):
        """청크들을 벡터 DB에 저장"""
        print("💾 벡터 DB에 저장 중...")
        
        try:
            # 기존 데이터 삭제 (새로 시작)
            try:
                self.client.delete_collection(self.collection_name)
                self.collection = self.client.create_collection(
                    name=self.collection_name,
                    metadata={"description": "보험 약관 문서 벡터 저장소"}
                )
                print("🗑️ 기존 데이터 삭제 완료")
            except:
                pass
            
            # 청크들을 배치로 저장
            batch_size = 100
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]
                
                # 데이터 준비
                documents = [chunk["content"] for chunk in batch]
                metadatas = [chunk["metadata"] for chunk in batch]
                ids = [f"chunk_{i}_{j}" for j in range(len(batch))]
                
                # 벡터 DB에 추가
                self.collection.add(
                    documents=documents,
                    metadatas=metadatas,
                    ids=ids
                )
                
                print(f"   ✅ 배치 {i//batch_size + 1} 저장 완료 ({len(batch)}개 청크)")
            
            print(f"💾 총 {len(chunks)}개 청크 저장 완료")
            
        except Exception as e:
            print(f"❌ 벡터 DB 저장 오류: {str(e)}")
    
    def search_similar(self, query: str, top_k: int = 5) -> List[Dict]:
        """유사한 문서 검색"""
        try:
            results = self.collection.query(
                query_texts=[query],
                n_results=top_k
            )
            
            # 결과 정리
            search_results = []
            for i in range(len(results['documents'][0])):
                search_results.append({
                    "content": results['documents'][0][i],
                    "metadata": results['metadatas'][0][i],
                    "distance": results['distances'][0][i]
                })
            
            return search_results
            
        except Exception as e:
            print(f"❌ 검색 오류: {str(e)}")
            return []
    
    def get_collection_info(self):
        """컬렉션 정보 조회"""
        try:
            count = self.collection.count()
            print(f"📊 컬렉션 정보:")
            print(f"   - 이름: {self.collection_name}")
            print(f"   - 총 문서 수: {count}")
            
            return count
            
        except Exception as e:
            print(f"❌ 컬렉션 정보 조회 오류: {str(e)}")
            return 0


## 실행

아래 셀에서 Google Drive의 JSON 파일 경로를 설정하고 실행하세요.

**경로 설정 방법:**
- Google Drive의 JSON 파일 경로: `/content/drive/MyDrive/폴더명/파일명.json`
- 예시: `/content/drive/MyDrive/processed_data/약관_pages.json`


In [ ]:
print("📘 벡터 저장소 구축 시스템")
print("=" * 60)

# Google Drive에서 JSON 파일 경로 설정
# 아래 경로를 본인의 Google Drive 파일 경로로 수정하세요
json_file = "/content/drive/MyDrive/processed_data/약관_pages.json"  # 예시 경로

# 벡터 DB 저장 위치 설정 (Google Drive에 저장할지 선택)
# True: Google Drive에 저장, False: Colab 작업 디렉토리에 저장
save_to_drive = True

if save_to_drive:
    # Google Drive에 저장
    chroma_db_path = "/content/drive/MyDrive/chroma_db"
else:
    # Colab 작업 디렉토리에 저장
    chroma_db_path = "./chroma_db"

if not os.path.exists(json_file):
    print(f"❌ 처리된 데이터 파일이 없습니다: {json_file}")
    print("💡 먼저 pdf_preprocessor.ipynb를 실행하여 JSON 파일을 생성하세요.")
    print(f"   또는 Google Drive의 올바른 경로로 수정하세요.")
else:
    # 2. 벡터 저장소 초기화 (Google Drive 경로 지정)
    vector_manager = VectorStore(
        collection_name="insurance_terms", 
        api_key=api_key,
        chroma_db_path=chroma_db_path
    )
    
    # 3. 데이터 로드
    pages_data = vector_manager.load_processed_data(json_file)
    if not pages_data:
        print("❌ 데이터를 로드할 수 없습니다.")
    else:
        # 4. 텍스트 청킹
        chunks = vector_manager.process_all_pages(pages_data)
        if not chunks:
            print("❌ 청크를 생성할 수 없습니다.")
        else:
            # 5. 벡터 DB에 저장
            vector_manager.store_in_vector_db(chunks)
            
            # 6. 저장소 정보 확인
            vector_manager.get_collection_info()
            
            print(f"\n🎉 벡터 저장소 구축 완료!")
            print(f"📁 저장 위치: {chroma_db_path}")
            print(f"📚 컬렉션: {vector_manager.collection_name}")


## 검색 테스트 (선택사항)

벡터 DB에 저장된 데이터를 검색해볼 수 있습니다.


In [ ]:
# 검색 테스트 (위에서 vector_manager가 생성된 경우에만 실행)
# 검색어를 변경하여 테스트하세요
query = "보험금 지급"

try:
    results = vector_manager.search_similar(query, top_k=3)
    
    print(f"🔍 검색어: '{query}'")
    print(f"📊 검색 결과: {len(results)}개\n")
    
    for i, result in enumerate(results, 1):
        print(f"--- 결과 {i} ---")
        print(f"페이지: {result['metadata']['page']}")
        print(f"거리: {result['distance']:.4f}")
        print(f"내용 (첫 200자): {result['content'][:200]}...")
        print()
except NameError:
    print("⚠️ vector_manager가 초기화되지 않았습니다. 위의 셀을 먼저 실행하세요.")
except Exception as e:
    print(f"❌ 검색 오류: {str(e)}")
